In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
np.set_printoptions(precision = 3)

In [2]:
jokes_df = pd.read_csv("jester-data-1.csv",header=None)
jokes_df = jokes_df.drop([0],axis=1)
jokes_df.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [3]:
#split the data into training and validation set
Partialdata,validation = train_test_split(jokes_df,test_size=0.1, random_state = 0)
validation.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
12590,3.69,-2.52,-0.15,-3.01,4.95,-1.46,-1.21,8.69,3.54,-2.91,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
12237,8.83,-8.06,8.50,8.20,7.62,7.62,6.31,8.59,9.03,8.64,...,99.00,99.00,-7.23,99.00,8.54,8.74,7.72,7.77,8.20,8.74
14382,-1.60,8.45,-6.99,1.07,5.15,5.53,-2.28,-1.65,1.70,-0.68,...,6.89,8.64,6.07,6.07,1.46,5.53,4.13,7.82,5.05,7.43
21596,0.87,1.50,-2.18,-4.76,-4.13,5.00,3.69,-0.63,-6.36,-3.98,...,0.63,6.07,1.07,3.45,2.23,2.57,4.56,5.73,5.68,2.57
168,99.00,2.23,99.00,99.00,-5.34,-2.09,-5.39,-4.95,99.00,8.06,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [4]:
# test set equalls the validation set with 99 rating
test = pd.DataFrame(validation)
test = test[test !=99.0]
test = test.dropna()
test.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
14382,-1.60,8.45,-6.99,1.07,5.15,5.53,-2.28,-1.65,1.70,-0.68,...,6.89,8.64,6.07,6.07,1.46,5.53,4.13,7.82,5.05,7.43
21596,0.87,1.50,-2.18,-4.76,-4.13,5.00,3.69,-0.63,-6.36,-3.98,...,0.63,6.07,1.07,3.45,2.23,2.57,4.56,5.73,5.68,2.57
17327,0.58,-2.33,-0.24,7.23,8.74,6.80,5.10,-9.42,-0.73,-4.71,...,0.10,2.82,6.55,3.06,4.71,8.30,-0.10,8.06,8.35,3.11
5224,7.72,8.59,-8.64,8.25,6.94,6.80,-4.03,-5.34,-3.16,3.98,...,-1.94,-9.08,7.96,7.62,6.99,9.27,9.27,-9.85,7.33,9.22
4423,7.14,-5.78,4.71,-0.87,8.59,-6.12,-6.17,-5.10,-5.58,6.70,...,-6.26,-6.26,-9.56,8.50,-6.89,-6.36,-5.68,1.55,-9.13,3.16


In [5]:
val = test.applymap(lambda x: 99)
val.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
14382,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
21596,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
17327,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
5224,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99
4423,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99


In [6]:
#feature generation for jokes
latent_item_features = np.random.random((val.shape[1],2))
latent_item_features.shape

(100, 2)

In [7]:
val = val.values
val

array([[99, 99, 99, ..., 99, 99, 99],
       [99, 99, 99, ..., 99, 99, 99],
       [99, 99, 99, ..., 99, 99, 99],
       ...,
       [99, 99, 99, ..., 99, 99, 99],
       [99, 99, 99, ..., 99, 99, 99],
       [99, 99, 99, ..., 99, 99, 99]])

In [8]:
latent_user_preferences = np.random.random((val.shape[0],2))
latent_user_preferences

array([[0.641, 0.735],
       [0.316, 0.525],
       [0.185, 0.338],
       ...,
       [0.006, 0.858],
       [0.13 , 0.218],
       [0.173, 0.695]])

In [11]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 100000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = test[user_id][item_id]
                if(not np.isnan(rating)):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if(iteration%10000 == 0 ):
            print (mse)

In [12]:
sgd()

KeyError: 0